In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [3]:
from config import DATA_DIR
ml_file = PROJECT_ROOT / DATA_DIR / "processed" / "features_for_ml.csv"

In [4]:
ml_df = pd.read_csv(ml_file, parse_dates=["Date"])

In [5]:
ml_df.describe()

,Date,rolling_14d_adjusted,price_available,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,wind_speed_10m_max,et0_fao_evapotranspiration,temperature_2m_max_30d_lag,...,et0_fao_evapotranspiration_120d_lag,et0_fao_evapotranspiration_180d_lag,heatwave_flag,coldwave_flag,heavy_rain_flag,temp_missing,USD_TL,trend,trends_missing,is_trend_zero
count,8769,8769.000000,8769.0,8769.000000,8769.000000,8769.000000,8769.000000,8769.000000,8769.000000,8769.000000,...,8769.000000,8769.000000,8769.000000,8769.000000,8769.000000,8769.0,8769.000000,4776.000000,4776.0,4776.000000
mean,2012-08-28 23:45:23.092712960,2.829628,1.0,17.114084,10.531144,3.428692,3.099704,13.969997,2.386388,17.280467,...,2.483747,2.440992,0.000114,0.057247,0.000798,0.0,6.794190,66.270472,0.0,0.175042
min,1999-12-26 00:00:00,0.730755,1.0,-2.800000,-10.800000,0.000000,0.000000,4.300000,0.180000,3.850000,...,1.011083,1.246722,0.000000,0.000000,0.000000,0.0,0.535502,0.000000,0.0,0.000000
25%,2005-12-26 00:00:00,2.192070,1.0,12.000000,5.400000,0.000000,0.000000,10.700000,1.270000,12.293333,...,1.618333,1.742167,0.000000,0.000000,0.000000,0.0,1.424333,27.807069,0.0,0.000000
50%,2012-04-06 00:00:00,2.764137,1.0,17.600000,10.400000,0.300000,0.300000,13.000000,2.160000,16.973333,...,2.464417,2.456500,0.000000,0.000000,0.000000,0.0,1.791467,49.184661,0.0,0.000000
75%,2019-11-29 00:00:00,3.363210,1.0,22.800000,16.300000,4.000000,3.500000,16.200000,3.420000,22.840000,...,3.339083,3.109167,0.000000,0.000000,0.000000,0.0,5.953000,83.319809,0.0,0.000000
max,2025-12-31 00:00:00,7.769791,1.0,35.800000,25.500000,78.700000,78.700000,53.700000,6.740000,29.780000,...,4.290583,3.861833,1.000000,1.000000,1.000000,0.0,42.939500,1087.689728,0.0,1.000000
std,NaN,1.207779,0.0,6.963901,6.844228,6.422489,6.017589,4.789735,1.336889,6.138736,...,0.893603,0.705018,0.010679,0.232327,0.028244,0.0,10.445633,68.401890,0.0,0.380043


In [6]:
ml_df.columns

Index(['Date', 'rolling_14d_adjusted', 'price_available', 'temperature_2m_max',
       'temperature_2m_min', 'precipitation_sum', 'rain_sum',
       'wind_speed_10m_max', 'et0_fao_evapotranspiration',
       'temperature_2m_max_30d_lag', 'temperature_2m_max_60d_lag',
       'temperature_2m_max_90d_lag', 'temperature_2m_max_120d_lag',
       'temperature_2m_max_180d_lag', 'temperature_2m_min_30d_lag',
       'temperature_2m_min_60d_lag', 'temperature_2m_min_90d_lag',
       'temperature_2m_min_120d_lag', 'temperature_2m_min_180d_lag',
       'precipitation_sum_30d_lag', 'precipitation_sum_60d_lag',
       'precipitation_sum_90d_lag', 'precipitation_sum_120d_lag',
       'precipitation_sum_180d_lag', 'rain_sum_30d_lag', 'rain_sum_60d_lag',
       'rain_sum_90d_lag', 'rain_sum_120d_lag', 'rain_sum_180d_lag',
       'wind_speed_10m_max_30d_lag', 'wind_speed_10m_max_60d_lag',
       'wind_speed_10m_max_90d_lag', 'wind_speed_10m_max_120d_lag',
       'wind_speed_10m_max_180d_lag', 'et0_fao_e

In [7]:
target_col = 'rolling_14d_adjusted'

drop_cols = [
    'Date',
    target_col,
    'trend',
    'trends_missing',
    'is_trend_zero'
]

In [8]:
X = ml_df.drop(columns=drop_cols)
y = ml_df[target_col]

In [9]:
train_mask = ml_df['Date'] < '2019-01-01'

X_train = X.loc[train_mask]
y_train = y.loc[train_mask]

X_test = X.loc[~train_mask]
y_test = y.loc[~train_mask]

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6318, 46) (2451, 46) (6318,) (2451,)


In [11]:
rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    min_samples_leaf=20,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",300
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",20
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples

In [12]:
y_pred_rf = rf.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Performance")
print("RMSE:", rmse_rf)
print("MAE:", mae_rf)
print("R²:", r2_rf)

Random Forest Performance
RMSE: 1.3370635650834362
MAE: 0.9385014165061156
R²: -0.7979463984723956


In [13]:
rf_importance = (
    pd.Series(rf.feature_importances_, index=X.columns)
    .sort_values(ascending=False)
)

rf_importance.head(15)

USD_TL                                 0.492390
precipitation_sum_90d_lag              0.068249
wind_speed_10m_max_120d_lag            0.061556
rain_sum_180d_lag                      0.055685
wind_speed_10m_max_180d_lag            0.049483
precipitation_sum_180d_lag             0.046943
rain_sum_30d_lag                       0.027312
rain_sum_120d_lag                      0.024978
rain_sum_90d_lag                       0.022298
precipitation_sum_120d_lag             0.021161
et0_fao_evapotranspiration_30d_lag     0.013120
temperature_2m_min_30d_lag             0.012456
wind_speed_10m_max_60d_lag             0.010763
wind_speed_10m_max_90d_lag             0.009269
et0_fao_evapotranspiration_180d_lag    0.008530
dtype: float64